# Reinforcement Learning: Policy Gradient
Un exemple simple de *policy gradient* sur l'environnement OpenAI Gym `LunarLander-v2`

In [1]:
import tensorflow as tf
import numpy as np
import gym

### On définit d'abord quelques constantes.

`LEARNING_RATE`: classique, contrôle la taille des pas de la descente de gradient.

`BATCH_SIZE`: Nombre d'épisodes (trajectoires) qu'on génère entre chaque étape d'apprentissage.

`EPOCHS`: Nombre de fois qu'on répète l'étape d'apprentissage.

`MAX_STEPS`: Permet de limiter la durée d'un épisode, on stoppe les épisodes trop long.

`MIN_REWARD`: De même, si un épisode atteint une récompense si basse, on le stoppe.

`GAMMA`: Permet de régler l'importance d'une récompense immédiate par rapport à une même récompense dans le futur. Une récompense dans $n$ transitions à partir d'un état donné sera multiplié par $\gamma^n$ pour l'estimation de la récompense future de cet état.

`HIDDEN_LAYER_WIDTH`: Largeur de notre petit réseau de neurone qui va estimer la stratégie.

In [2]:
LEARNING_RATE = 0.05
BATCH_SIZE = 15
EPOCHS = 2000
MAX_STEPS = 600
MIN_REWARD = -350
GAMMA = 0.99
HIDDEN_LAYER_WIDTH = 15

### Définition de l'environnement

In [3]:
env = gym.make("LunarLander-v2")

s_size = env.observation_space.shape[0]
a_size = env.action_space.n

print("Number of actions: {}".format(a_size))
print("Shape of the state: {}".format(env.observation_space.shape))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Number of actions: 4
Shape of the state: (8,)


### Définition de notre policy

In [4]:
tf.reset_default_graph()

# Placeholder qui contiendra une liste de vecteurs d'états
with tf.variable_scope("placeholders"):
    states_plh = tf.placeholder(shape=[None, s_size], dtype=tf.float32, name="states")

# Couche intermédiaire du réseau de neurones (activation ReLU)
with tf.variable_scope("hidden_layer"):
    weights_l1 = tf.get_variable("weights", shape=[s_size, HIDDEN_LAYER_WIDTH], initializer=tf.contrib.layers.xavier_initializer())
    biases_l1 = tf.get_variable("biases", shape=[HIDDEN_LAYER_WIDTH], initializer=tf.zeros_initializer)
    out_l1 = tf.nn.relu(tf.matmul(states_plh, weights_l1) + biases_l1)

# Couche de sortie
with tf.variable_scope("outputs"):
    weights_l2 = tf.get_variable("weights", shape=[HIDDEN_LAYER_WIDTH, a_size], initializer=tf.contrib.layers.xavier_initializer(1))
    biases_l2 = tf.get_variable("biases", shape=[a_size], initializer=tf.zeros_initializer)

    # Liste de vecteurs de probabilités (probabilité de chaque action)
    policy_probs = tf.nn.softmax(tf.matmul(out_l1, weights_l2) + biases_l2)
    

### Définition de l'entrainement

In [5]:
with tf.variable_scope("placeholders"):
    # Au moment de l'entrainement, on a besoin de spécifier la liste des actions prise,
    # et la liste des récompenses futures obtenues (ajustées avec GAMMA)
    actions_plh = tf.placeholder(shape=[None,], dtype=tf.int32, name="actions")
    rewards_plh = tf.placeholder(shape=[None,], dtype=tf.float32, name="rewards")

# Depuis les vecteurs de probabilités, sélectionne uniquement les valeurs qui ont mené à l'action associée
responsible_probs = tf.reduce_sum(policy_probs * tf.one_hot(actions_plh, depth=a_size), axis=1)

# Objectif à maximiser, pondéré par les recompenses données
objective = tf.reduce_mean(tf.log(responsible_probs) * rewards_plh)

optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
train_op = optimizer.minimize(-objective)

### Discounted rewards
A partir du vecteur des récompenses, on calcule pour chaque élément la future récompense obtenue à partir de cet état. Plus une récompense est dans le future, plus elle est réduite (en fonction de $\gamma$).

In [6]:
def discounted_reward(rewards):
    ret = []
    running_r = 0
    rewards -= np.mean(rewards)
    for r in reversed(rewards):
        running_r = running_r * GAMMA + r
        ret.append(running_r)
        
    ret = np.asarray(list(reversed(ret)))
    return ret

### Génération de trajectoire
Pour chaque état, on utilise notre réseau de neurones pour obtenir une distribution des probabilités de chaque action. On choisit l'action à prendre en suivant cette distribution. A chaque pas, on stocke l'état précédent, l'action effectuée, l'état suivant, et la récompense obtenue dans une liste.

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

def run_trajectory(render=False):
    s = env.reset()
    
    total_reward = 0
    traj = []
    
    for _ in range(MAX_STEPS):
        probs = sess.run(policy_probs, feed_dict={states_plh: [s]})[0]
        a = np.random.choice(range(a_size), p=probs)
        
        s1, r, d, _ = env.step(a)
        if render:
            env.render()
        
        traj.append([s, a, s1, r])
        
        total_reward += r
        
        s = s1
        
        if total_reward < MIN_REWARD:
            break
    
    return (np.array(traj), total_reward)

### Lancement de l'apprentissage

In [ ]:
# saver.restore(sess, "./save-ckpt/ckpt")

for epoch in range(EPOCHS):
    s = env.reset()
    trajectories = []
    rewards = []
    total_rewards = []
    
    for batch in range(BATCH_SIZE):
        traj, r = run_trajectory(False)
        trajectories.append(traj)
        
        discounted = discounted_reward(traj[:, 3])
        rewards.append(discounted)
        
        total_rewards.append(r)
        
    # Accumule les trajectoires
    all_trajs = np.vstack(trajectories)
    
    # Accumule les récompenses, puis les centre et les normalise (permet de réduire la variance de la policy)
    all_rewards = np.hstack(rewards)
    all_rewards -= np.mean(all_rewards)
    all_rewards /= np.std(all_rewards)
        
    feed_dict = {
                states_plh: np.vstack(all_trajs[:, 0]),
                actions_plh: all_trajs[:,1],
                rewards_plh: all_rewards
    }
    
    # Lance une étape de descente de gradient
    _ = sess.run([train_op], feed_dict=feed_dict)
    
    saver.save(sess, "save-ckpt/ckpt")
    
    print("Epoch {} finished. Mean reward: {}".format(epoch, np.mean(total_rewards)))
          
            
    

### Visualisation du résultat

In [ ]:
sess = tf.Session()
saver.restore(sess, "./save-ckpt/ckpt")

for _ in range(100):
    s = env.reset()
    while True:
        probs = sess.run(policy_probs, feed_dict={states_plh: [s]})[0]
        a = np.random.choice(range(a_size), p=probs)
        env.render()
        s, r, d, _ = env.step(a)

        if d:
            break

### Bonus: Génère un GIF ! (actions aléatoires)

In [ ]:
# Generate a GIF for random actions

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 

NUM_RUNS = 10

# Police dans le même répertoire
font = ImageFont.truetype("Ubuntu.ttf", 40)

i = 0
for _ in range(5):
    s = env.reset()
    running_r = 0
    
    while True:
        img = env.render(mode='rgb_array')
        
        # Ecriture du score
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        draw.text((20, 20),"Score: {:5.2f}".format(running_r),(255,255,255),font=font)
        
        img.save('frame_{:04}.jpg'.format(i))
        env.render()
        
        probs = sess.run(policy_probs, feed_dict={states_plh: [s]})[0]
        a = np.random.choice(range(a_size), p=probs)
        s, r, d, _ = env.step(a)
        running_r += r
        i += 1

        if d:   
            break
    